TODO:
* experiment with narrower sampling of thetas / less trials / fixed presentation time
* analysis with train set / test set
* reaction times

In [ ]:
# %pip install -U -r requirements.txt

In [ ]:
%load_ext watermark
%watermark -i -h -m -v -p numpy,MotionClouds,manim,pandas,matplotlib,scipy

In [ ]:
import os
from datetime import datetime
now = datetime.now()
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


In [23]:
# https://laurentperrinet.github.io/sciblog/posts/2020-08-09-nesting-jupyter-runs.html
def has_parent():
    """
    https://stackoverflow.com/questions/48067529/ipython-run-magic-n-switch-not-working
    
    Return True if this notebook is being run by calling
    %run in another notebook, False otherwise.
    """
    try:
        __file__
        # __file__ has been defined, so this notebook is 
        # being run in a parent notebook
        return True

    except NameError:
        # __file__ has not been defined, so this notebook is 
        # not being run in a parent notebook
        return False
def do_verb():
    return not has_parent()

verbose = do_verb()
if verbose : print('__name__=', __name__, '\n ', verbose, ', I am a running this notebook directly. ')

__name__= __main__ 
  True , I am a running this notebook directly. 


# experiment 1 (aka pilot): one B_sf / some B_thetas / many thetas

In [24]:
experiment_name = 'pilot'

In [25]:
# %rm -fr img_pilot

In [26]:

data_folder = f'img_{experiment_name}'

if not(os.path.isfile(f'{data_folder}/parameters.json')):
    os.makedirs(data_folder, exist_ok=True)

    print('Initializing')
    print(50*'.-*')

    # parameters 
    import MotionClouds as mc

    N_B_theta = 9
    N_B_sf = 1
    N_repet = 2
    N_thetas = 12

    B_thetas = np.pi/3 * np.linspace(0, 1, N_B_theta+1)[1:]
    print('B_thetas = ', B_thetas)
    B_sfs = [mc.B_sf] #* np.logspace(-1, -1, N_B_sf, base=2)
    print('B_sfs = ', B_sfs)
    theta_max = np.pi/8
    thetas = np.linspace(-theta_max, theta_max, N_thetas)

    print(50*'.-*')
    parameters = pd.DataFrame(columns=['i_trial', 'theta', 'i_theta', 'B_theta', 'i_B_theta', 'B_sf', 'i_B_sf', 'seed', 'fname'])
    # generate all clouds
    import imageio
    def generate_random_cloud(i_trial, theta, B_theta, B_sf, seed, downscale = 1):
        # fname = f'{data_folder}/theta_{theta}_B_theta_{B_theta}_B_sf_{B_sf}_seed_{seed}.png'
        fname = f'{data_folder}/{i_trial}.png'
        if not os.path.isfile(fname):
            fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)
            mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0., B_sf=B_sf,
                                    B_V=0, theta=np.pi/2-theta, B_theta=B_theta)
            im = mc.random_cloud(mc_i, seed=seed)
            im = (mc.rectif(im) * 255).astype('uint8')
            imageio.imwrite(fname, im[:, :, 0])
        return fname


    all_conditions = [(i_repet, i_theta, i_B_theta, i_B_sf) 
                    for i_repet in range(N_repet) 
                    for i_theta in range(N_thetas) 
                    for i_B_theta in range(N_B_theta) 
                    for i_B_sf in range(N_B_sf)]
    N_total_trials = len(all_conditions)
    ind = np.random.permutation(N_total_trials)

    seed = 2024
    np.random.seed(seed)
    # parameters = []
    for i_trial in range(N_total_trials):
        i_repet, i_theta, i_B_theta, i_B_sf = all_conditions[ind[i_trial]]

        fname = generate_random_cloud(i_trial, thetas[i_theta], 
                                B_theta=B_thetas[i_B_theta], 
                                B_sf=B_sfs[i_B_sf], 
                                seed=seed+i_trial)
        # parameters.append({'fname':fname, 'theta': thetas[i_theta], 'B_theta': B_thetas[i_B_theta], 'B_sf': B_sfs[i_B_sf], 'seed': seed+i_trial, 'i_trial': i_trial})
        parameters.loc[i_trial] = [i_trial, thetas[i_theta], i_theta, B_thetas[i_B_theta], i_B_theta, B_sfs[i_B_sf], i_B_sf, seed+i_trial, fname]
        print(f"          {{stimulus: '{fname}', on_finish: function() {{jsPsych.setProgressBar({i_trial/N_total_trials:.4f});}}}},")

    parameters.to_json(f'{data_folder}/parameters.json', orient = 'records', indent=4)
else:
    parameters = pd.read_json(f'{data_folder}/parameters.json')
if verbose: 
    from IPython.display import display, HTML
    display(HTML(parameters.to_html()))

,i_trial,theta,B_theta,i_B_theta,B_sf,seed,fname
0,0,0.392699,0.116355,0,0.1,2024,img_pilot_folder/0.png
1,1,-0.249899,0.465421,3,0.1,2025,img_pilot_folder/1.png
2,2,-0.107100,1.047198,8,0.1,2026,img_pilot_folder/2.png
3,3,0.321299,0.698132,5,0.1,2027,img_pilot_folder/3.png
4,4,0.178500,0.581776,4,0.1,2028,img_pilot_folder/4.png
5,5,0.392699,0.232711,1,0.1,2029,img_pilot_folder/5.png
6,6,-0.249899,0.814487,6,0.1,2030,img_pilot_folder/6.png
7,7,0.321299,0.581776,4,0.1,2031,img_pilot_folder/7.png
8,8,-0.178500,0.349066,2,0.1,2032,img_pilot_folder/8.png
9,9,-0.321299,0.698132,5,0.1,2033,img_pilot_folder/9.png


In [27]:
# # # for i_trial in range(len(parameters)):
# # #     print(f"i_trial: {i_trial}, B_theta: {parameters.loc[i_trial, 'B_theta']}")
# # # B_thetas = np.pi/3 * np.linspace(0, 1, N_B_theta+1)[1:]
# # rev_B_thetas  = {}
# # for i_B_theta, B_theta in enumerate(B_thetas):
# #     rev_B_thetas[B_thetas[i_B_theta]] = i_B_theta

# # rev_B_thetas
# for i_trial in range(len(parameters)):
#     B_theta = parameters.loc[i_trial, 'B_theta']
#     # print(f"i_trial: {i_trial}, B_theta: {B_theta}, {rev_B_thetas[B_theta]}")

# parameters_i = pd.DataFrame(columns=['i_trial', 'theta', 'B_theta',  'i_B_theta',  'B_sf', 'seed', 'fname'])
# for i_trial in range(len(parameters)):
#     B_theta = parameters.loc[i_trial, 'B_theta']
#     parameters_i.loc[i_trial] = [i_trial, parameters.loc[i_trial, 'theta'], B_theta, rev_B_thetas[B_theta], parameters.loc[i_trial, 'B_sf'], parameters.loc[i_trial, 'seed'], parameters.loc[i_trial, 'fname']]
# parameters_i

# parameters_i.to_json(f'{data_folder}/parameters.json', orient = 'records', indent=4)

In [31]:
# i_B_theta_trials = np.array(parameters['i_B_theta'])
# np.histogram(i_B_theta_trials, bins = 9)


In [35]:
# B_thetas = np.sort(np.array(parameters['B_theta'].unique()))*180/np.pi
# B_thetas, np.pi/3 * np.linspace(0, 1, 9+1)[1:]*180/np.pi

In [ ]:
# %rm -fr img_pilot

#   analysing data

In [ ]:
# %pip install osfclient

In [ ]:
import glob


In [ ]:
# import osfclient
# osfclient.cli.init?

Collect file names:

In [ ]:

filenames = []
for fname in glob.glob(f'osfstorage-archive/{experiment_name}*json'):
    filenames.append(fname)
# filenames

## time elapsed per session

In [ ]:
for fname in filenames:
    df = pd.read_json(fname)
    if verbose: print(f"{fname}: total seconds elapsed {np.array(df[df['trial_type']=='image-swipe-response']['time_elapsed'])[-1]/1000:.0f}")
    # print(df)

Remove such that are obviously cancelled sessions:

In [ ]:
filenames_valid = []

minimal_time_threshold = 50

for fname in filenames:
    df = pd.read_json(fname)
    if np.array(df[df['trial_type']=='image-swipe-response']['time_elapsed'])[-1]/1000 > minimal_time_threshold:
        filenames_valid.append(fname)
        
if verbose: 
    print(filenames_valid)

In [ ]:
if verbose: print('Number of valid sessions:', len(filenames_valid), ', Average time', np.mean([np.array(pd.read_json(fname)[pd.read_json(fname)['trial_type']=='image-swipe-response']['time_elapsed'])[-1]/1000 for fname in filenames_valid]))

# Collect data


Let's finally gather data

In [ ]:
responses = {}

for i_fname, fname in enumerate(filenames_valid):
    session = fname.replace(f'osfstorage-archive/{experiment_name}-', '').replace('-data.json', '')
    df = pd.read_json(fname)
    df_data = df[df['trial_type']=='image-swipe-response'][['trial_index', 'stimulus', 'swipe_response', 'keyboard_response', 'rt', 'response_source']]
    y = np.array(((df_data['swipe_response'] == 'right') + (df_data['keyboard_response'] == 'arrowright')))*1.
    responses[session] = y

In [ ]:
if verbose: responses.keys()

## other imports

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# # https://pytorch.org/docs/main/generated/torch.nn.BCELoss.html
criterion = torch.nn.BCELoss(reduction="mean")
# # https://pytorch.org/docs/main/generated/torch.nn.BCEWithLogitsLoss.html#torch.nn.BCEWithLogitsLoss
criterion_logits = torch.nn.BCEWithLogitsLoss(reduction="mean")

In [ ]:

if torch.cuda.is_available():  # To use the GPU with CUDA (Win/Linux)
    device = "cuda"
elif torch.backends.mps.is_available():  # To use the GPU on MacOS
    device = "mps"
    device = "cpu"  # Fallback to use the CPU - my benchmark shows it's actually faster
else:
    device = "cpu"  # Fallback to use the CPU

In [ ]:
# %whos